# exp044_ens

exp042(lgb)とexp044(cat)のアンサンブル

In [1]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from collections import defaultdict

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
from catboost import CatBoost, Pool

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
@dataclass
class Cfg:
    mode = "local_cv" # "local_cv" or "kaggle_inf" 
    exp_name = "exp044"
    input_dir = "/mnt/predict-student-performance-from-game-play/input/"
    output_dir = "/mnt/predict-student-performance-from-game-play/output/"
    prep_dir = "/mnt/predict-student-performance-from-game-play/prep/"
    seed = 42
    n_splits = 5
    best_threshold = 0.630 # local_cvの結果を入れる
cfg = Cfg()

if cfg.mode == "local_cv":
    os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name), exist_ok=True)
    os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name, "cache"), exist_ok=True)

elif cfg.mode == "kaggle_inf":
    import jo_wilder

In [3]:
def calc_metrics(oof):
    logloss = log_loss(oof["correct"], oof["pred"])

    # find best th
    scores = []; thresholds = []
    best_score = 0; best_threshold = 0

    for threshold in np.arange(0.4,0.81,0.01):
        preds = (oof["pred"].values>threshold).astype(int)
        m = f1_score(oof["correct"].values, preds, average='macro')   
        scores.append(m)
        thresholds.append(threshold)
        if m>best_score:
            best_score = m
            best_threshold = threshold
    print("logloss", format(logloss, ".6f"))
    print("best_score", format(best_score, ".6f"))
    print("best_threshold", format(best_threshold, ".3f"))

    # Q別スコア
    print("---"*10)
    for q in range(18):
        q = q + 1
        preds = (oof[oof["question"]==q]["pred"].values>threshold).astype(int)
        m = f1_score(oof[oof["question"]==q]["correct"].values, preds, average='macro')
        print(f"Q{q} : F1 = {format(m, '.6f')}")
    return best_threshold

In [4]:
oof42 = pd.read_csv(cfg.output_dir + "exp042/oof.csv.gz")
oof44 = pd.read_csv(cfg.output_dir + "exp044/oof.csv.gz")

In [6]:
oof42["pred"] = (oof42["pred"] + oof44["pred"]) / 2
calc_metrics(oof42)

logloss 2.914045
best_score 0.688695
best_threshold 0.580
------------------------------
Q1 : F1 = 0.653374
Q2 : F1 = 0.509657
Q3 : F1 = 0.522434
Q4 : F1 = 0.660614
Q5 : F1 = 0.480255
Q6 : F1 = 0.645602
Q7 : F1 = 0.630999
Q8 : F1 = 0.520432
Q9 : F1 = 0.630271
Q10 : F1 = 0.421203
Q11 : F1 = 0.542371
Q12 : F1 = 0.575702
Q13 : F1 = 0.426350
Q14 : F1 = 0.599992
Q15 : F1 = 0.395206
Q16 : F1 = 0.531361
Q17 : F1 = 0.521238
Q18 : F1 = 0.498738


0.5800000000000002